<a href="https://colab.research.google.com/github/shu65/blog-pytorch-notebooks/blob/main/pytorch_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [19]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [22]:
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [23]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [24]:
# training parameters 
batch_size = 64
test_batch_size = 1000
epochs = 2
lr = 1.0
gamma = 0.7
seed = 1
log_interval = 100

In [25]:
torch.manual_seed(seed)
device = torch.device("cuda")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
common_kwargs = {'num_workers': 1, 'pin_memory': True, 'shuffle': True}
train_kwargs.update(common_kwargs)
test_kwargs.update(common_kwargs)

transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                              ])
dataset1 = datasets.MNIST('./data', train=True, download=True,
                          transform=transform)
dataset2 = datasets.MNIST('./data', train=False,
                          transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
  train(model, device, train_loader, optimizer, epoch, log_interval)
  test(model, device, test_loader)
  scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.331472
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.399083
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.326026
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.137404
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.194266
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.060524
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.108608
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.200231
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.040291
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.063507

Test set: Average loss: 0.0594, Accuracy: 9818/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.083315
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.329067
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.067283
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.051089
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.087292
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.222237
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.097125
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.016038
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.163205
T